# Conversor de Arquivos TXT para CSV

Este notebook converte todos os arquivos .txt do diretório `raw_data` para o formato .csv no diretório `converted_csv_data`.

Os arquivos contêm dados de vento com colunas separadas por ponto e vírgula (;) que serão convertidas para o formato CSV padrão com vírgulas.

## Arquivos a serem convertidos:
- Dir.txt (Direção do vento)
- U.txt (Componente U do vento)
- V.txt (Componente V do vento) 
- W.txt (Componente W do vento)
- WS.txt (Velocidade do vento)

#### 1. Importa as libs.

In [8]:
import pandas as pd
import os
from pathlib import Path
import glob

#### 2. Configura o caminho dos diretórios

In [9]:
# Definir diretórios
raw_data_dir = Path("../raw_data")
csv_data_dir = Path("../converted_csv_data")

# Verificar se o diretório de origem existe
if raw_data_dir.exists():
    print(f"✓ Diretório {raw_data_dir} encontrado")
else:
    print(f"✗ Diretório {raw_data_dir} não encontrado")

✓ Diretório ../raw_data encontrado


#### 3. Cria um caminho de output

In [10]:
# Criar diretório de destino se não existir
csv_data_dir.mkdir(exist_ok=True)

if csv_data_dir.exists():
    print(f"✓ Diretório {csv_data_dir} criado/encontrado com sucesso")
else:
    print(f"✗ Erro ao criar diretório {csv_data_dir}")

# Listar conteúdo atual do diretório de destino
print(f"\nConteúdo atual do diretório {csv_data_dir}:")
csv_files = list(csv_data_dir.glob("*.csv"))
if csv_files:
    for file in csv_files:
        print(f"  - {file.name}")
else:
    print("  (vazio)")

✓ Diretório ../converted_csv_data criado/encontrado com sucesso

Conteúdo atual do diretório ../converted_csv_data:
  - U.csv
  - Dir.csv
  - WS.csv
  - V.csv
  - W.csv


#### 4. Função de conversão de arquivos

In [11]:
import numpy as np

def convert_txt_to_csv(txt_file_path, csv_file_path):
    """
    Converte um arquivo TXT com delimitador ';' para CSV com delimitador ','
    Trata valores ausentes para serem detectados como NaN pelo pandas
    
    Args:
        txt_file_path (Path): Caminho para o arquivo TXT de origem
        csv_file_path (Path): Caminho para o arquivo CSV de destino
    
    Returns:
        bool: True se a conversão foi bem-sucedida, False caso contrário
    """
    try:
        # Lista abrangente de valores que devem ser interpretados como NaN
        na_values = ['', ' ', '  ', '\t', '\n', '\r', 
                     'N/A', 'NA', 'n/a', 'na', 'N/a', 'n/A',
                     'NULL', 'null', 'Null', 'None', 'none', 'NONE',
                     'NaN', 'nan', 'NAN', 'Nan',
                     '-', '--', '---', '____', '....',
                     '9999', '-9999', '999999', '-999999',
                     '999.9', '-999.9', '9999.9', '-9999.9',
                     '999.99', '-999.99', '9999.99', '-9999.99']
        
        # Ler o arquivo TXT com delimitador ';' e tratar valores ausentes
        df = pd.read_csv(txt_file_path, 
                        sep=';', 
                        encoding='utf-8',
                        na_values=na_values,
                        keep_default_na=True)
        
        # Garantir que células vazias sejam convertidas para NaN
        df = df.replace(['', ' ', '  '], np.nan)
        
        # Converter strings que representam valores ausentes em NaN
        df = df.replace(na_values, np.nan)
        
        # Verificar se há valores ausentes após a conversão
        missing_count = df.isna().sum().sum()
        total_cells = df.shape[0] * df.shape[1]
        missing_percentage = (missing_count / total_cells) * 100
        
        # Salvar como CSV com delimitador ',' e valores nulos como 'NaN'
        df.to_csv(csv_file_path, index=False, encoding='utf-8', na_rep='NaN')
        
        print(f"✓ {txt_file_path.name} → {csv_file_path.name}")
        print(f"  └── {missing_count} valores NaN detectados ({missing_percentage:.2f}% do total)")
        return True
        
    except Exception as e:
        print(f"✗ Erro ao converter {txt_file_path.name}: {str(e)}")
        return False

print("Função de conversão atualizada - valores nulos serão salvos como 'NaN' no CSV!")

Função de conversão atualizada - valores nulos serão salvos como 'NaN' no CSV!


#### 5. Pega todos o .txt

In [12]:
# Encontrar todos os arquivos .txt no diretório raw_data
txt_files = list(raw_data_dir.glob("*.txt"))

print(f"Arquivos TXT encontrados ({len(txt_files)}):")
for txt_file in txt_files:
    print(f"  - {txt_file.name}")

if not txt_files:
    print("Nenhum arquivo .txt encontrado no diretório raw_data")

Arquivos TXT encontrados (5):
  - V.txt
  - W.txt
  - WS.txt
  - U.txt
  - Dir.txt


#### 6. Converte cada .txt em um .csv correspondente

In [13]:
# Converter cada arquivo TXT para CSV
successful_conversions = 0
failed_conversions = 0

print("Iniciando conversão dos arquivos...\n")

for txt_file in txt_files:
    # Criar o nome do arquivo CSV correspondente
    csv_file_name = txt_file.stem + ".csv"
    csv_file_path = csv_data_dir / csv_file_name
    
    # Converter arquivo
    if convert_txt_to_csv(txt_file, csv_file_path):
        successful_conversions += 1
    else:
        failed_conversions += 1

print(f"\n--- Resumo da Conversão ---")
print(f"✓ Conversões bem-sucedidas: {successful_conversions}")
print(f"✗ Conversões com erro: {failed_conversions}")
print(f"Total de arquivos processados: {len(txt_files)}")

Iniciando conversão dos arquivos...

✓ V.txt → V.csv
  └── 132658 valores NaN detectados (11.76% do total)
✓ V.txt → V.csv
  └── 132658 valores NaN detectados (11.76% do total)
✓ W.txt → W.csv
  └── 115011 valores NaN detectados (10.19% do total)
✓ W.txt → W.csv
  └── 115011 valores NaN detectados (10.19% do total)
✓ WS.txt → WS.csv
  └── 132658 valores NaN detectados (11.76% do total)
✓ WS.txt → WS.csv
  └── 132658 valores NaN detectados (11.76% do total)
✓ U.txt → U.csv
  └── 132658 valores NaN detectados (11.76% do total)
✓ U.txt → U.csv
  └── 132658 valores NaN detectados (11.76% do total)
✓ Dir.txt → Dir.csv
  └── 132658 valores NaN detectados (11.76% do total)

--- Resumo da Conversão ---
✓ Conversões bem-sucedidas: 5
✗ Conversões com erro: 0
Total de arquivos processados: 5
✓ Dir.txt → Dir.csv
  └── 132658 valores NaN detectados (11.76% do total)

--- Resumo da Conversão ---
✓ Conversões bem-sucedidas: 5
✗ Conversões com erro: 0
Total de arquivos processados: 5


#### 7. verificação dos resultados

In [14]:
# Verificar arquivos CSV criados
csv_files = list(csv_data_dir.glob("*.csv"))

print("=== VERIFICAÇÃO DOS RESULTADOS ===\n")

print(f"Arquivos CSV criados ({len(csv_files)}):")
for csv_file in csv_files:
    file_size = csv_file.stat().st_size
    print(f"  - {csv_file.name} ({file_size:,} bytes)")

print(f"\n--- Comparação de Estrutura ---")

# Verificar estrutura de um arquivo como exemplo
if csv_files:
    example_csv = csv_files[0]
    print(f"\nExemplo: {example_csv.name}")
    
    # Carregar e mostrar informações básicas
    df_csv = pd.read_csv(example_csv)
    print(f"  Dimensões: {df_csv.shape[0]} linhas × {df_csv.shape[1]} colunas")
    print(f"  Colunas: {list(df_csv.columns)[:5]}..." if len(df_csv.columns) > 5 else f"  Colunas: {list(df_csv.columns)}")
    
    # Mostrar primeiras linhas
    print(f"\n  Primeiras 3 linhas:")
    print(df_csv.head(3).to_string(index=False))

print(f"\nConversão concluída! Todos os arquivos estão disponíveis em: {csv_data_dir}")

=== VERIFICAÇÃO DOS RESULTADOS ===

Arquivos CSV criados (5):
  - U.csv (7,061,801 bytes)
  - Dir.csv (6,285,692 bytes)
  - WS.csv (6,217,749 bytes)
  - V.csv (6,778,500 bytes)
  - W.csv (6,372,903 bytes)

--- Comparação de Estrutura ---

Exemplo: U.csv
  Dimensões: 41786 linhas × 27 colunas
  Colunas: ['DT', 'SODAR-30', 'SODAR-40', 'SODAR-50', 'SODAR-60']...

  Primeiras 3 linhas:
                 DT  SODAR-30  SODAR-40  SODAR-50  SODAR-60  SODAR-70  SODAR-80  SODAR-90  SODAR-100  SODAR-110  SODAR-120  SODAR-130  SODAR-140  SODAR-150  SODAR-160  SODAR-170  SODAR-180  SODAR-190  SODAR-200  SODAR-210  SODAR-220  SODAR-230  SODAR-240  SODAR-250  SODAR-260  SODAR-270  SODAR-280
2021/11/10 17:20:00     -1.16     -2.49     -2.81     -2.70     -2.95     -2.98     -3.11      -3.20      -3.20      -3.10      -2.97      -2.87      -2.80      -2.73      -2.80      -2.79      -2.76      -2.74      -2.74      -2.79      -2.74      -2.82      -2.92      -2.81      -2.74      -3.01
2021/11/10 17:30: